In [2]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubun

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Review-Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [5]:
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [38]:
vine_df=vine_df.withColumn("helpful_votes",df.helpful_votes.cast('int'))\
.withColumn("total_votes",df.total_votes.cast('int'))\
.withColumn("star_rating",df.star_rating.cast('int'))\
.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [39]:
vine_df.describe

<bound method DataFrame.describe of DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]>

In [40]:
clean_review=vine_df.filter("total_votes>=20").select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [41]:
clean_review.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [42]:
clean_review.describe()

DataFrame[summary: string, review_id: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string]

In [10]:
from pyspark.sql.functions import round
clean_review.withColumn("helpful_votesrate",round((clean_review["helpful_votes"]/clean_review["total_votes"]),2)).show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votesrate|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|              1.0|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|              1.0|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|             0.92|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|              1.0|
| R36LII9IITE17|          1|           22|         24|   N|                Y|             0.92|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|             0.92|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|             0.98|
|R2FYQBKCC1XG4M|          5|           2

In [80]:
review_df=clean_review.filter((clean_review["helpful_votes"]/clean_review["total_votes"])> 0.5).select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [81]:
review_df.describe()

DataFrame[summary: string, review_id: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string]

In [83]:
vine_review=review_df.filter(review_df["vine"]=="Y").select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [84]:
other_review=review_df.filter(review_df["vine"]=="N").select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])


In [86]:
from pyspark.sql.functions import round, count, countDistinct

In [87]:
#Vine review analysis
total_vine_review=vine_review.select(count("star_rating")).show()
vine_star_count=vine_review.groupBy("star_rating").count().show()

+------------------+
|count(star_rating)|
+------------------+
|                21|
+------------------+

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          3|    1|
|          5|   11|
|          4|    8|
|          2|    1|
+-----------+-----+



In [77]:
percent_vine_5_start=11/21
print("Vine review 5 stars precentage is: " + str(percent_vine_5_start))

Vine review 5 stars precentage is: 0.5238095238095238


In [88]:
#Other review analysis
total_other_review=other_review.select(count("star_rating")).show()
other_star_count=other_review.groupBy("star_rating").count().show()

+------------------+
|count(star_rating)|
+------------------+
|              7685|
+------------------+

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          1| 1231|
|          3|  601|
|          5| 4443|
|          4|  932|
|          2|  478|
+-----------+-----+



In [89]:
percent_other_5_start=4443/7685
print("Vine review 5 stars precentage is: " + str(percent_other_5_start))

Vine review 5 stars precentage is: 0.5781392322706571
